In [2]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata

dataset = pd.read_csv("/content/movie_reviews.csv.bz2")
dataset.info()

In [ ]:
dataset.head()

In [ ]:
dataset['review'][1]

In [ ]:
Char = "áü"

In [15]:
def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
  return stripped_text

In [ ]:
strip_html_tags(dataset['review'][20])

In [17]:
def remove_accented_characters(text):
  text = unicodedata.normalize("NFKD",text).encode("ascii",'ignore').decode('utf-8','ignore')
  return text

In [18]:
remove_accented_characters("áü")

'au'

In [32]:
def pre_process_corpus(texts):
  preprocessed_text = []
  for doc in tqdm.tqdm(texts):
    doc = strip_html_tags(doc)
    #doc = doc.translate(doc.maketrans("\n\t\r",'   '))
    doc = doc.lower()
    doc = remove_accented_characters(doc)
    #Do remove contractions as well and check the results
    doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,re.I|re.A)
    doc = re.sub(r' +',' ',doc)
    doc = doc.strip()
    #doc = "".join(doc)
    preprocessed_text.append(doc)
  return preprocessed_text

In [34]:
reviews  = dataset['review'].values
sentiment = dataset['sentiment'].values

train_reviews = reviews[:35000]
train_sentiment = sentiment[:35000]

test_reviews = reviews[35000:]
test_sentiment = sentiment[35000:]

In [ ]:
pre_process_corpus([dataset['review'][20]])

In [35]:
%%time
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

  2%|▏         | 859/35000 [00:00<00:11, 2888.77it/s]<ipython-input-15-46650b581e77>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text,'html.parser')
100%|██████████| 15000/15000 [00:05<00:00, 2881.04it/s]

CPU times: user 18.1 s, sys: 178 ms, total: 18.3 s
Wall time: 18.6 s


### Feature Extraction

In [36]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer(binary = False,ngram_range=(1,2))

In [37]:
cv_train_features = cv.fit_transform(norm_train_reviews)

In [38]:
cv_train_features

<35000x2169706 sparse matrix of type '<class 'numpy.int64'>'
	with 11934545 stored elements in Compressed Sparse Row format>

In [39]:
cv.get_feature_names_out()

array(['00', '00 agent', '00 for', ..., 'zzzzzzzzzzzzzzzzzz if',
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz',
       'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz 8ooops'], dtype=object)

In [40]:
cv_test_features = cv.transform(norm_test_reviews)

In [44]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty = 'l2',max_iter=500,C=1)

lr.fit(cv_train_features,train_sentiment)

lr_bow_predictions = lr.predict(cv_test_features)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

labels = ['negative','positive']
print(classification_report(test_sentiment,lr_bow_predictions))
pd.DataFrame(confusion_matrix(test_sentiment, lr_bow_predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.91      0.90      0.90      7490
    positive       0.90      0.91      0.90      7510

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



,negative,positive
negative,6749,741
positive,708,6802


In [49]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Tv = TfidfVectorizer(use_idf= True,ngram_range=(1,2))

Tv_train_features = Tv.fit_transform(norm_train_reviews)
Tv_test_features = Tv.transform(norm_test_reviews)

lr = LogisticRegression(penalty = 'l2',max_iter=500,C=1)

lr.fit(Tv_train_features,train_sentiment)

lr_tfidf_predictions = lr.predict(Tv_test_features)

In [50]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

labels = ['negative','positive']
print(classification_report(test_sentiment,lr_tfidf_predictions))
pd.DataFrame(confusion_matrix(test_sentiment, lr_tfidf_predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.90      0.88      0.89      7490
    positive       0.89      0.91      0.90      7510

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



,negative,positive
negative,6606,884
positive,705,6805


In [51]:
#Homework..
#Add contractions see if there is any change in the result
#Try Random Forest and Look at the results
#One-Hot encoding..